<a href="https://colab.research.google.com/github/vaibhavlokunde/AI-ML/blob/main/BreastCancerPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import sklearn

In [4]:
if torch.cuda.is_available():
  device="cuda"
else:
  device="cpu"
device

'cpu'

In [2]:
from sklearn.datasets import load_breast_cancer
data=load_breast_cancer()
X=data.data
y=data.target

In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42)

In [8]:
len(X_train),len(X_test),len(y_train),len(y_test)

(455, 114, 455, 114)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [10]:
X_train=torch.from_numpy(X_train).type(torch.float)
X_test=torch.from_numpy(X_test).type(torch.float)
y_train=torch.from_numpy(y_train).type(torch.float)
y_test=torch.from_numpy(y_test).type(torch.float)

In [21]:
class BreastCancerModelV1(nn.Module):
  def __init__(self,in_size,hidden_units,out_size):
    super().__init__()
    self.fc1=nn.Linear(in_size,hidden_units)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_units,out_size)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    x=self.sigmoid(x)
    return x


In [22]:
input_size=X_train.shape[1]
hidden_size=64
output_size=1

In [26]:
model_1=BreastCancerModelV1(input_size,hidden_size,output_size)
model_1

BreastCancerModelV1(
  (fc1): Linear(in_features=30, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [27]:
loss_fn=nn.BCELoss()
optimizer=torch.optim.Adam(params=model_1.parameters(),lr=0.001)

In [28]:
def accuracy(y_true,y_pred):
  correct=torch.eq(y_true,y_pred).sum().item()
  acc=correct/len(y_pred)
  return acc

In [30]:
epochs=100

for epoch in range(epochs):
  model_1.train()

  y_pred=model_1(X_train).squeeze()

  loss=loss_fn(y_pred,y_train)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  with torch.no_grad():
    predicted=y_pred.round()
    correct=torch.eq(y_train,predicted).sum().item()
    acc=correct/len(y_pred)

  if epoch%10==0:
    print(f"Epoch:{epoch} | Loss:{loss} | Test Accuracy:{acc}")

Epoch:0 | Loss:0.6320874691009521 | Test Accuracy:0.8263736263736263
Epoch:10 | Loss:0.4958104193210602 | Test Accuracy:0.9274725274725275
Epoch:20 | Loss:0.3895378112792969 | Test Accuracy:0.9384615384615385
Epoch:30 | Loss:0.3070855140686035 | Test Accuracy:0.9384615384615385
Epoch:40 | Loss:0.2462954968214035 | Test Accuracy:0.9362637362637363
Epoch:50 | Loss:0.20282717049121857 | Test Accuracy:0.9406593406593406
Epoch:60 | Loss:0.17184151709079742 | Test Accuracy:0.9472527472527472
Epoch:70 | Loss:0.1493539810180664 | Test Accuracy:0.9604395604395605
Epoch:80 | Loss:0.13243110477924347 | Test Accuracy:0.9626373626373627
Epoch:90 | Loss:0.11920434981584549 | Test Accuracy:0.967032967032967


In [34]:
model_1.eval()
with torch.inference_mode():
  y_pred=model_1(X_test).squeeze()
  loss=loss_fn(y_pred,y_test)
  predicted=y_pred.round()
  acc=accuracy(y_test,predicted)

print(f"Model accuracy is:{acc*100}%")

Model accuracy is:97.36842105263158%
